In [3]:
import sys

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import numpy as np

from ignite.engine import create_supervised_trainer
from ignite.engine.engine import Events
from ignite.handlers import ModelCheckpoint

# assumes the framework is found here, change as necessary
sys.path.append("..")

import monai


monai.config.print_config()

MONAI version: 0.0.1
Python version: 3.8.1 (default, Jan  8 2020, 22:29:32)  [GCC 7.3.0]
Numpy version: 1.18.1
Pytorch version: 1.4.0
Ignite version: 0.3.0


In [5]:
lr = 1e-3

net = monai.networks.nets.UNet(
    dimensions=2,
    in_channels=1,
    num_classes=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)


def fake_loss(y_pred,y):
    return (y_pred[0]+y).sum()


def fake_data_stream():
    while True:
        yield torch.rand((10,1,64,64)),torch.rand((10,1,64,64))
        
        
# 1 GPU
opt = torch.optim.Adam(net.parameters(), lr)
trainer=monai.engine.create_multigpu_supervised_trainer(net,opt,fake_loss,[torch.device('cuda:0')])
trainer.run(fake_data_stream(),2,2)

# all GPUs
opt = torch.optim.Adam(net.parameters(), lr)
trainer=monai.engine.create_multigpu_supervised_trainer(net,opt,fake_loss,None)
trainer.run(fake_data_stream(),2,2)

# CPU
opt = torch.optim.Adam(net.parameters(), lr)
trainer=monai.engine.create_multigpu_supervised_trainer(net,opt,fake_loss,[])
trainer.run(fake_data_stream(),2,2)

State:
	iteration: 4
	epoch: 2
	epoch_length: 2
	max_epochs: 2
	output: 26133.853515625
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'generator'>
	seed: 12